- title: Compiling libtensorflow in Ubuntu 18.04 to use it from Scala
- author: Oscar Vargas Torres
- date: 2018-08-21
- category: Setup
- tags: TensorFlow

# Repositories and dependencies

[Installation instructions](http://platanios.org/tensorflow_scala/installation.html) provided for tensorflow_scala <cite data-cite="727676/X4D3I4KT"></cite> do not work at the moment of writing (2018-08-21). It may be related to differences in the supported hardware the native library was compiled for.

So, it was necessary for me to compile libtensorflow.so from source:

```
git clone https://github.com/tensorflow/tensorflow.git
cd tensorflow
git checkout r1.9
```

I installed the Intel Distribution for Python, with instructions for `intelpython3_core` from [here](https://software.intel.com/en-us/articles/using-intel-distribution-for-python-with-anaconda). 

Next, it is time to configure the tensorflow build:

```
yes "" | TF_NEED_MKL=1 TF_DOWNLOAD_MKL=0 \
         MKL_INSTALL_PATH=/home/oscarvarto/anaconda3/envs/idp ./configure 
``` 

It is necessary to [install bazel](https://docs.bazel.build/versions/master/install-ubuntu.html#install-with-installer-ubuntu), and then run the actual build. I chose the following flags according to the supported instruction sets supported by my processor (use `cat /proc/cpuinfo` for that).

```
bazel build --cxxopt=-D_GLIBCXX_USE_CXX11_ABI=0 \
            --config=mkl -c opt --copt=-mavx --copt=-mfma --copt=-msse4.2 \
            //tensorflow:libtensorflow.so
```

# First Scala TensorFlow program

## sbt dependencies

```scala
name := "HelloTensorFlow"

version := "0.1"

scalaVersion := "2.12.6"

libraryDependencies ++= Seq(
  "org.platanios" %% "tensorflow" % "0.2.3"
)
```

## Hellow TensorFlow

```scala
import org.platanios.tensorflow.api._

object HelloTensorFlow extends App {
  val t1 = Tensor( 1.2, 4.5)
  val t2 = Tensor(-0.2, 1.1)
  val sum = t1 + t2
  println(s"sum = ${sum.summarize(flattened=true)}") 
}
```

**sbt must be able to find the native libraries compiled before**. Therefore, `$LD_LIBRARY_PATH` must include the `bazel-bin/tensorflow` folder inside the tensorflow repository:
```
# ~/.zshenv (assuming zsh shell)
LD_LIBRARY_PATH=`~/gitRepos/tensorflow/bazel-bin/tensorflow
export LD_LIBRARY_PATH
```

<div class="cite2c-biblio"></div>